In [1]:
import requests
import json
import time
import traceback
import pymysql
import pandas as pd
from datetime import datetime

CONNECT TO AWS RDS

In [2]:
dbikes = pymysql.Connect(
    host = "dbikes.ccike2q3zkya.eu-west-1.rds.amazonaws.com",
    user = "admin",
    passwd = "admin2022",
    database = "dbikes")

CREATE THREE TABLES

static_station columns: number, name, bike stands, lat, lng

dynamic_station columns: number, last_update, available_bike_stands, available_bikes

dynamic_weather columns: number, lat, lng, dt, temp, pressure, wind_speed, humidity

In [3]:
cur = dbikes.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS static_station(number integer, name varchar(45), bike_stands integer, lat float, lng float)")
cur.execute("CREATE TABLE IF NOT EXISTS dynamic_station(number integer, last_update integer, available_bike_stands integer, available_bikes integer)")
cur.execute("CREATE TABLE IF NOT EXISTS dynamic_weather(number integer, lat float, lng float, dt integer, temp float, pressure float, wind_speed float, humidity float)")

0

Insert the static_station table

In [9]:
# for i in range(len(data)):
#     station_row = data[i]
#     static_station_insert_query = "INSERT INTO static_station(number, name, bike_stands, lat, lng) VALUES(%s, %s, %s, %s, %s)"
#     static_station_data = (int(station_row.get("number")),
#                            station_row.get("name"),
#                            int(station_row.get("bike_stands")),
#                            float(station_row.get("position").get("lat")),
#                            float(station_row.get("position").get("lng")))
#     cur.execute(static_station_insert_query, static_station_data)
# dbikes.commit()           

Method to request data from JDCecaux API and Openweather API

In [4]:
#dublin bike station api
APIKEY = "a4ae2329e4585bbd13dcf83332d04b69a88fb904" 
NAME = "Dublin" 
STATIONS_URI = "https://api.jcdecaux.com/vls/v1/stations"
api_response = requests.get(STATIONS_URI, params={"apiKey": APIKEY, "contract": NAME})
data = json.loads(api_response.text)

#openweather api
WEATHER_APIKEY = "eeec159d31a816c2152dbf05ba6e0076"
WEATHER_URL = "http://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}"

In [14]:
def InsertData2Tables(station):
    for i in range(0,len(station)):
        station_row = station[i]
        weather_response = requests.get(WEATHER_URL.format(station_row.get("position").get("lat"), station_row.get("position").get("lng"), WEATHER_APIKEY))
        weather = json.loads(weather_response.text)
#         insert the station data
        station_insert_query = "INSERT INTO dynamic_station(number, last_update, available_bike_stands, available_bikes) VALUES(%s, %s, %s, %s)"
        station_data = (int(station_row.get("number")),
                        float(station_row.get("last_update")),
                        int(station_row.get("available_bike_stands")),
                        int(station_row.get("available_bikes")))
        cur.execute(station_insert_query, station_data)
        
#         insert the weather data
#         here still in this rage because each station match each weather, which is the same in number
        weather_insert_query = "INSERT INTO dynamic_weather(number, lat, lng, dt, temp, pressure, wind_speed, humidity) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"
        weather_data = (int(station_row.get("number")),
                        float(weather["coord"].get("lat")),
                        float(weather["coord"].get("lon")),
                        int(weather["dt"]),
                        float(weather["main"].get("temp")),
                        float(weather["main"].get("pressure")),
                        float(weather["wind"].get("speed")),
                        float(weather["main"].get("humidity")))
        cur.execute(weather_insert_query, weather_data)
    dbikes.commit()

In [15]:
def ContinuousGetData():
    while True:
        try:
            api_response = requests.get(STATIONS_URI, params={"apiKey": APIKEY, "contract": NAME})
            data = json.loads(api_response.text)
            InsertData2Tables(data)
            
#             sleep every 5 minutes
            time.sleep(5*60)
    
        except:
#             hit for problems
            print(traceback.format_exc())

In [ ]:
ContinuousGetData()

In [ ]:

# create a static_df
static_table = pd.DataFrame(
    columns=("number", "name", "last_update", "available_bike_stands", "available_bikes", "bike_stands","lat","lng", "status"))

# create table to store the dynamic weather data 
dynamic_weather_table = pd.DataFrame(
    columns=("number", "lat", "lng", "dt", "temp", "pressure", "wind_speed", "humidity"))

#fill the dataframe
for i in range(len(data)):
    weather_response = requests.get(WEATHER_URL.format(data[i].get("position").get("lat"), data[i].get("position").get("lng"), WEATHER_APIKEY))
    weather_data = json.loads(weather_response.text)
#     fill static_dataframe
    static_table.loc[i] = [int(data[i].get("number")),data[i].get("name"), float(data[i].get("last_update")), 
                           int(data[i].get("available_bike_stands")),
                           int(data[i].get("available_bikes")),
                           int(data[i].get("bike_stands")),
                           data[i].get("position").get("lat"),data[i].get("position").get("lng"),data[i].get("status")]
    
#      fill weather_dataframe
    dynamic_weather_table.loc[i] = [int(data[i].get("number")),
                                    float(weather_data["coord"].get("lat")), float(weather_data["coord"].get("lon")),
                                    int(weather_data["dt"]),
                                    float(weather_data["main"].get("temp")),
                                    float(weather_data["main"].get("pressure")),
                                    float(weather_data["wind"].get("speed")),
                                    float(weather_data["main"].get("humidity"))]

In [189]:
#  convert the timestrip to datetime
static_table["last_update"] = pd.to_datetime(static_table["last_update"], unit = "ms")
#  add weekday to table as a parameter
static_table["weekday"] = static_table["last_update"].map(lambda x: x.weekday())
static_table.head()

,number,name,last_update,available_bike_stands,available_bikes,bike_stands,lat,lng,status,weekday
0,42,SMITHFIELD NORTH,2022-02-14 00:40:44,7,23,30,53.349562,-6.278198,OPEN,0
1,30,PARNELL SQUARE NORTH,2022-02-14 00:43:58,14,6,20,53.353462,-6.265305,OPEN,0
2,54,CLONMEL STREET,2022-02-14 00:50:14,18,15,33,53.336021,-6.262980,OPEN,0
3,108,AVONDALE ROAD,2022-02-14 00:45:03,16,19,35,53.359405,-6.276142,OPEN,0
4,56,MOUNT STREET LOWER,2022-02-14 00:41:03,23,17,40,53.337960,-6.241530,OPEN,0


In [190]:
#  convert the timestrip to datetime
dynamic_weather_table["dt"] = pd.to_datetime(dynamic_weather_table["dt"], unit = "s")
#  add weekday to table as a parameter
dynamic_weather_table["weekday"] = dynamic_weather_table["dt"].map(lambda x: x.weekday())
dynamic_weather_table.head()

,number,lat,lng,dt,temp,pressure,wind_speed,humidity,weekday
0,42.0,53.3496,-6.2782,2022-02-14 00:50:30,278.61,993.0,6.69,91.0,0
1,30.0,53.3535,-6.2653,2022-02-14 00:50:30,278.58,993.0,6.69,91.0,0
2,54.0,53.3400,-6.2600,2022-02-14 00:46:18,278.59,993.0,6.69,91.0,0
3,108.0,53.3594,-6.2761,2022-02-14 00:50:30,278.52,993.0,6.69,91.0,0
4,56.0,53.3380,-6.2415,2022-02-14 00:50:30,278.55,993.0,6.69,91.0,0


In [191]:
dynamic_weather_table

,number,lat,lng,dt,temp,pressure,wind_speed,humidity,weekday
0,42.0,53.3496,-6.2782,2022-02-14 00:50:30,278.61,993.0,6.69,91.0,0
1,30.0,53.3535,-6.2653,2022-02-14 00:50:30,278.58,993.0,6.69,91.0,0
2,54.0,53.3400,-6.2600,2022-02-14 00:46:18,278.59,993.0,6.69,91.0,0
3,108.0,53.3594,-6.2761,2022-02-14 00:50:30,278.52,993.0,6.69,91.0,0
4,56.0,53.3380,-6.2415,2022-02-14 00:50:30,278.55,993.0,6.69,91.0,0
5,6.0,53.3438,-6.2672,2022-02-14 00:43:34,278.62,993.0,6.69,91.0,0
6,18.0,53.3341,-6.2654,2022-02-14 00:50:30,278.57,993.0,6.69,91.0,0
7,32.0,53.3443,-6.2504,2022-02-14 00:50:30,278.61,993.0,6.69,91.0,0
8,52.0,53.3400,-6.2600,2022-02-14 00:46:18,278.59,993.0,6.69,91.0,0
9,48.0,53.3478,-6.2442,2022-02-14 00:50:30,278.61,993.0,6.69,91.0,0


In [ ]:
dynamic_weather_table.d

In [192]:
static_table.columns

Index(['number', 'name', 'last_update', 'available_bike_stands',
       'available_bikes', 'bike_stands', 'lat', 'lng', 'status', 'weekday'],
      dtype='object')

In [228]:
# static_table = static_table.drop(["lat","lng","status"],axis = 1)
# del static_table['weekday']
merge_table = static_table.merge(dynamic_weather_table,left_on = "number",right_on = "number")

# assume that canpacity only effect by hour regardless of the holiday
# convert the date to hour
merge_table["hour"] = pd.to_datetime(merge_table["last_update"], format='%Y-%M-%D %H:%M:%S').dt.hour
regression_table = pd.DataFrame(merge_table,columns = ["number","available_bike_stands","available_bikes","bike_stands","temp",
                                                      "pressure","wind_speed","humidity","weekday","hour"])


In [231]:
regression_table.dtypes

number                     int64
available_bike_stands      int64
available_bikes            int64
bike_stands                int64
temp                     float64
pressure                 float64
wind_speed               float64
humidity                 float64
weekday                  float64
hour                       int64
dtype: object

In [230]:
# covert the object type to int64
regression_table["number"] = regression_table["number"].astype("int64")
regression_table["available_bike_stands"] = regression_table["available_bike_stands"].astype("int64")
regression_table["available_bikes"] = regression_table["available_bikes"].astype("int64")
regression_table["bike_stands"] = regression_table["bike_stands"].astype("int64")
regression_table.corr()

,number,available_bike_stands,available_bikes,bike_stands,temp,pressure,wind_speed,humidity,weekday,hour
number,1.000000,0.357747,0.136767,0.562979,-0.092720,NaN,-0.152502,NaN,NaN,NaN
available_bike_stands,0.357747,1.000000,-0.597897,0.571667,0.109414,NaN,-0.225620,NaN,NaN,NaN
available_bikes,0.136767,-0.597897,1.000000,0.315737,-0.037541,NaN,0.162949,NaN,NaN,NaN
bike_stands,0.562979,0.571667,0.315737,1.000000,0.092407,NaN,-0.100463,NaN,NaN,NaN
temp,-0.092720,0.109414,-0.037541,0.092407,1.000000,NaN,-0.007839,NaN,NaN,NaN
pressure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wind_speed,-0.152502,-0.225620,0.162949,-0.100463,-0.007839,NaN,1.000000,NaN,NaN,NaN
humidity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weekday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here, because of the lack of data, there is no Correlation coefficient of humidity, weekday and hour.


All of the above data is the same value, so the result is NaN

In [196]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [250]:
# Split the data into train and test sets
# Take a third (random) data samples as test data, rest as training data
# If we want to get the same train/test split every time we run, we can set the random_state variable to a fixed value

def train_test(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
    
    #need to reset the index to allow contatenation with predicted values otherwise not joining on same index...
    #Training Data reset index
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    
    # Testing Data reset index
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)

    return X_train, X_test, y_train, y_test

In [245]:
def Prediction(X_train, X_test, y_train, y_test):
    #Prediction on Trainig  set 
    #model = LinearRegression()
    
    rfc = RandomForestRegressor(n_estimators=100, max_features='auto', oob_score=True, random_state=1)
    
    rfc.fit(X_train,y_train)
    
    importance = pd.DataFrame({'feature': X_train.columns, 'importance':rfc.feature_importances_})
    print(importance.sort_values('importance', ascending=False))
    
    #Prediction on Train Data

    rfc_predictions_train = rfc.predict(X_train)
    df_true_vs_rfc_predicted = pd.DataFrame({'ActualClass': y_train, 'PredictedClass': rfc_predictions_train})
    print(df_true_vs_rfc_predicted.head(20))
    
    #Prediction on test Data
    rfc_predictions_test = rfc.predict(X_test)
    df_true_vs_rfc_predicted_test = pd.DataFrame({'ActualClass': y_test, 'PredictedClass': rfc_predictions_test})
    df_true_vs_rfc_predicted_test.head(20)
    
    print("\nrfc.oob_score_",rfc.oob_score_,"\n")
    return rfc

In [246]:
station_number = regression_table["number"].drop_duplicates().values.tolist()
station_number = sorted(station_number)
print(station_number)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117]


In [247]:
cont_features = ["available_bike_stands","available_bikes","bike_stands","temp",
                 "pressure","wind_speed","humidity","weekday","hour"]

In [248]:
def bikeAvailability():
    for _ in station_number:
        df_new = regression_table[regression_table.number==_]
        X = df_new[cont_features]
        y = df_new.available_bikes
        
        print("\n~~~~~~ STATION NUMBER =",_,"~~~~~~\n")
        X_train, X_test, y_train, y_test = train_test(X,y)
       
        rfc = Prediction(X_train, X_test, y_train, y_test)
        
        with open('./PickelFiles/'+str(_)+'.pkl','wb') as write:
            pickle.dump(rfc,write)
    
    return

In [251]:
bikeAvailability()


~~~~~~ STATION NUMBER = 2 ~~~~~~



ValueError: With n_samples=1, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [264]:
cont_features = ["bike_stands","available_bikes","temp",
                 "pressure","wind_speed","humidity","hour"]
X_W = regression_table[cont_features]
y_W = regression_table.available_bike_stands
X_train_W, X_test_W, y_train_W, y_test_W = train_test(X_W,y_W)
X_W.dtypes

bike_stands          int64
available_bikes      int64
temp               float64
pressure           float64
wind_speed         float64
humidity           float64
hour                 int64
dtype: object

In [265]:
X_train_W

,bike_stands,available_bikes,temp,pressure,wind_speed,humidity,hour
0,40,7,278.60,993.0,6.69,91.0,0
1,33,15,278.59,993.0,6.69,91.0,0
2,30,25,278.53,993.0,6.69,91.0,0
3,40,7,278.61,993.0,6.69,91.0,0
4,30,21,278.60,993.0,6.69,91.0,0
5,40,16,278.61,993.0,6.69,91.0,0
6,40,29,278.52,993.0,6.69,91.0,0
7,25,11,278.58,993.0,6.69,91.0,0
8,40,29,278.61,993.0,6.69,91.0,0
9,40,20,278.64,993.0,6.69,91.0,0


In [266]:
rfc = RandomForestRegressor(n_estimators=50, max_features='auto', oob_score=True, random_state=1)
rfc.fit(X_train_W,y_train_W)

RandomForestRegressor(n_estimators=50, oob_score=True, random_state=1)

In [267]:
importance = pd.DataFrame({'feature': X_train_W.columns, 'importance':rfc.feature_importances_})
importance.sort_values('importance', ascending=False)

,feature,importance
1,available_bikes,0.532079
0,bike_stands,0.452267
2,temp,0.011838
4,wind_speed,0.003816
3,pressure,0.000000
5,humidity,0.000000
6,hour,0.000000


In [268]:
#Prediction on Train Data

rfc_predictions_train = rfc.predict(X_train_W)
df_true_vs_rfc_predicted = pd.DataFrame({'ActualClass': y_train_W, 'PredictedClass': rfc_predictions_train})
df_true_vs_rfc_predicted.head(50)

,ActualClass,PredictedClass
0,33,32.88
1,18,18.46
2,5,5.06
3,33,32.92
4,9,8.98
5,24,24.24
6,11,11.06
7,14,13.90
8,10,11.18
9,20,19.92


In [269]:
#Prediction on Test Data

rfc_predictions_test = rfc.predict(X_test_W)
df_true_vs_rfc_predicted_test = pd.DataFrame({'ActualClass': y_test_W, 'PredictedClass': rfc_predictions_test})
df_true_vs_rfc_predicted_test.head(20)

,ActualClass,PredictedClass
0,0,5.36
1,8,8.48
2,15,13.80
3,4,11.12
4,17,17.90
5,11,11.80
6,16,14.76
7,10,11.26
8,34,33.88
9,18,14.72


In [270]:
rfc.oob_score_

0.9671783043255752

In [1]:
conda install mysql-connector-python

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                               failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - mysql-connector-python -> python[version='>=2.7,<2.8.0a0|>=3.7,<3.8.0a0|>=3.8,<3.9.0a0|>=3.6,<3.7.0a0|>=3.5,<3.6.0a0']

Your python: python=3.9

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the pyth

In [2]:
pip install pymysql

     |████████████████████████████████| 43 kB 3.5 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [10]:
git clone https://github.com/invicta117/COMP20230_Data_Struc_and_Algorithms_Assignment_1.git

SyntaxError: invalid syntax (804964574.py, line 1)